# Intro

We'll now walk through the hidden-shift algorithm implementation in Qiskit for a four bit function with $s=0000$. We first set the number of qubits used in the experiment, and the hidden bit string $s$ to be found by the algorithm. The hidden bit string $s$ determines the circuit for the quantum oracle.

In [ ]:
# Delete me later.
'''# Importing Qiskit and IonQ accounts
from qiskit import IBMQ

# Loading your IBM Quantum and IonQ accounts
# IBMQ.save_account('token')
IBMQ.load_account()
ibmq_provider = IBMQ.get_provider('ibm-q-skku','yonsei-uni','yu-students')'''

In [ ]:
# Delete me later.
'''import numpy as np
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt'''

In [1]:
# initialization
import matplotlib.pyplot as plt

# importing Qiskit
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram

# import basic plot tools
from qiskit.visualization import plot_histogram

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
n = 4
s = 2

# Assigning oracles

Refer: https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html
QuantumCircuit(*regs, name=None, global_phase=0, metadata=None)

![circuit_fig](images/tentative_fig1.png)

In [ ]:
oracle_g = QuantumCircuit(int(n/2))
oracle_g.z(0)

shift_s = QuantumCircuit(n)
if s%2 == 1:
    shift_s.x(0)
if s%4 - s%2 == 2:
    shift_s.x(1)
if s%8 - s%4 == 4:
    shift_s.x(2)
if s%16 - s%8 == 8:
    shift_s.x(3)

oracle_f = QuantumCircuit(n)
oracle_f.compose(shift_s, inplace=True)
oracle_f.barrier()
oracle_f.compose(oracle_g, qubits=[0,1], inplace=True)
oracle_f.barrier()
oracle_f.cz(0,2)
oracle_f.cz(1,3)
oracle_f.barrier()
oracle_f.compose(shift_s, inplace=True)

oracle_sf = QuantumCircuit(n)
oracle_sf.compose(oracle_g, qubits=[2,3], inplace=True)
oracle_sf.barrier()
oracle_sf.cz(0,2)
oracle_sf.cz(1,3)

# Algorithm for hidden-shift problem



In [ ]:
circ = QuantumCircuit(n, n)

for i in range(n):
    circ.h(i)
circ.barrier()
circ.compose(oracle_f, inplace=True)
circ.barrier()
for i in range(n):
    circ.h(i)
circ.barrier()
circ.compose(oracle_sf, inplace=True)
circ.barrier()
for i in range(n):
    circ.h(i)
circ.barrier()
for i in range(n):
    circ.measure(i, i)

In [ ]:
circ.draw('mpl')

# Experiment with simulators

We can run the above circuit on the simulator.

In [ ]:
# use local simulator
aer_sim = Aer.get_backend('aer_simulator')
transpiled_circ = transpile(circ, aer_sim)
qobj = assemble(transpiled_circ, aer_sim)
results = aer_sim.run(qobj).result()
answer = results.get_counts()

plot_histogram(answer)

We can see that the result of the measurement is the hidden string 0010.